<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"><b>Exploring SAR Data and SAR Time Series Analysis using Jupyter Notebooks</b></font>

<br>
<font size="4"><b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" />
</font>

<font size="3"> This notebook will introduce you to the analysis of deep multi-temporal SAR image data stacks in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>We introduce the following data analysis concepts:</b>

- How to load your own SAR data into Jupyter Notebooks and create a time series stack 
- How to apply calibration constants to covert initial digital number (DN) data into calibrated radar cross section information.
- How to subset images and create a time series of your subset data.
- How to explore the time-series information in SAR data stacks for environmental analysis.

</font>

<hr>
<font face="Calibri" size="5" color="red"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://pandas.pydata.org/" target="_blank">Pandas</a></b> is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. </li>
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
    <li> The <b><a href="https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html" target="_blank">asf-hyp3 API</a></b> provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype"). </li>
<li><b><a href="https://www.scipy.org/about.html" target="_blank">SciPY</a></b> is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics. </li>

</font>

<font face="Calibri" size="3"> Our first step is to <b>import them:</b> </font>

In [ ]:
!pip install pyproj # this can be removed when pyproj is installed on jupyterHub

In [ ]:
import os # for chdir, getcwd, path.exists
import glob # for glob
import re # for match
import json # for loads
import math # for ceil
import datetime # for date
import pickle
import gc

from pyproj import Proj, transform
import pandas as pd # for DatetimeIndex
import gdal # for gdalbuildvr, gdalmerge, gdalwarp, gdal_translate, Open
from osgeo import gdal # for Info
import numpy as np # for copy, isnan, log10, ma.masked_where, max, mean, min, percentile, power, unique, var, where 
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches  # for Rectangle
from matplotlib import rc
from asf_hyp3 import API # for get_products, get_subscriptions, login
import scipy # for signal.savgol_filter()
import scipy.signal

from IPython.display import HTML

from asf_notebook import path_exists
from asf_notebook import download_hyp3_products
from asf_notebook import new_directory
from asf_notebook import earthdata_hyp3_login
from asf_notebook import remove_nan_filled_tifs
from asf_notebook import select_RTC_polarization

<font face="Calibri" size="3"><b>Setup matplotlib plotting</b> inside the notebook:</font>

In [ ]:
%matplotlib inline 

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Your Own Data Stack Into the Notebook </b> </font> 

<font size="3"> This notebook assumes that you've created your own data stack over your personal area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system <a href="http://hyp3.asf.alaska.edu/" target="_blank">HyP3</a>. HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This notebook expects <a href="https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf" target="_blank">Radiometric Terrain Corrected</a> (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP. Prefer a **unique orbit geometry** (ascending or descending) to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font> 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's End User License Agreement (EULA) applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the EULA in EarthData, you will need to navigate to <a href="https://earthdata.nasa.gov/" target="_blank">EarthData's home page</a> and complete that process.</b></font>
<br><br>
<b>Login to Earthdata:</b> </font> 

In [ ]:
api = earthdata_hyp3_login()

<hr>
<font face="Calibri" size="3"> Before we download anything, let's <b>first create a working directory for this analysis and change into it:</b> </font>

In [ ]:
path = "/home/jovyan/notebooks/SAR_Training/English/data_time_series_pickle_tile"
new_directory(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3"><b>Create a folder in which to download your RTC products.</b> </font>

In [ ]:
new_directory("rtc_products")
products_path = f"{path}/rtc_products"

<font face="Calibri" size="3"><b>Create a folder in which to store pickled data:</b> </font>

In [ ]:
new_directory(f"{path}/pickles")

<font face="Calibri" size="3"><b>Set a date range of products to download:</b> </font>

In [ ]:
date_range = [datetime.date(2018, 6, 1), datetime.date(2019, 1, 1)] #enter your date range here
direction = 'D' # enter a flight direction here (A or D)
#flight_path = 114 # enter a flight path

########## NOTE: Currently filtering by path and flight_direction does not work for InSAR products #########

# uncomment code below to download all products
#date_range = [None, None]
#direction = None
flight_path = None

<font face="Calibri" size="3"><b>Download the products associated with an existing RTC subscription.</b> </font>

In [ ]:
subscription_id = download_hyp3_products(
    api, products_path, start_date=date_range[0], end_date=date_range[1], flight_direction=direction, path=flight_path)

<hr>
<font face="Calibri" size="3"><b>Determine the subscription's process type</b>, which we need in order to determine the file paths to the tiffs.</font>

In [ ]:
subscription_info = api.get_subscription(subscription_id)
process_type = subscription_info['process_id']

<font face="Calibri" size="3"><b>Create a variable called <i>paths</i>, that holds the paths to the images</b>, which varies based on process type:</font>

In [ ]:
rtc_path = "rtc_products"
paths = select_RTC_polarization(process_type, rtc_path)
print(f"paths: {paths}")
polarization = paths.split('.')[0][-2:]
print(polarization)

<font face="Calibri" size="3"><b>Write a function to collect the product acquisition dates:</b></font>

In [ ]:
def get_dates(paths, split_index, date_start_index, date_end_index):
    dates = []
    pths = glob.glob(paths)
    for p in pths:
        date = p.split("_")[split_index][date_start_index:date_end_index]
        dates.append(date)
    dates.sort()
    return dates

<font face="Calibri" size="3"><b>Call get_dates() to collect the product acquisition dates:</b></font>

In [ ]:
dates = get_dates(paths, 5, 0, 8)
print(dates)

<hr>
<font face="Calibri" size="3"> You may notice duplicates in your acquisition dates. As HyP3 processes SAR data on a frame-by-frame basis, duplicates may occur if your area of interest is covered by two consecutive  image frames. In this case, two separate images are generated that need to be merged together before time series processing can commence. 
<br><br>
<b>Write functions to collect and print the paths of the tiffs:</b> </font>

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = !ls $paths | sort -t_ -k5,5
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face="Calibri" size="3"><b>Collect and print the paths of the tiffs:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths)

<hr>
<font face="Calibri" size="4"> <b>1.2 Fix multiple UTM Zone-related issues</b> <br>
<br>
<font face="Calibri" size="3">Fix multiple UTM Zone-related issues should they exist in your data set. If multiple UTM zones are found, the following code cells will identify the predominant UTM zone and reproject the rest into that zone. This step must be completed prior to merging frames or performing any analysis.</font>
<br><br>
<font face="Calibri" size="3"><b>Use gdal.Info to determine the UTM definition types and zones in each product:</b></font>

In [ ]:
utm_zones = []
utm_types = []
print('Checking UTM Zones in the data stack ...\n')
for p in tiff_paths:
    info = (gdal.Info(p, options = ['-json']))
    info = (json.loads(info))['coordinateSystem']['wkt']
    zone = info[(len(info)-8):(len(info)-3)]
    utm_zones.append(zone)
    typ = info[(len(info)-15):(len(info)-11)]
    utm_types.append(typ)
print(f"UTM Zones:\n {utm_zones}\n")
print(f"UTM Types:\n {utm_types}")

<font face="Calibri" size="3"><b>Identify the most commonly used UTM Zone in the data:</b></font>

In [ ]:
stack_info = {}
utm_unique, counts = np.unique(utm_zones, return_counts=True)
a = np.where(counts == np.max(counts))
stack_info.update({'utm': utm_unique[a][0]})
print(f"Predominate UTM Zone: {stack_info['utm']}")

<font face="Calibri" size="3"><b>Reproject images with errant UTMs to the predominate UTM:</b></font>

In [ ]:
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != stack_info['utm']] #makes list of indicies in utm_zones that need to be reprojected
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print('--------------------------------------------')
for k in reproject_indicies:
    temppath = tiff_paths[k].strip()
    _, product_name, tiff_name = temppath.split('/')
    cmd = f"gdalwarp -overwrite rtc_products/{product_name}/{tiff_name} rtc_products/{product_name}/r{tiff_name} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{stack_info['utm']}"
    
    #print(f"Calling the command: {cmd}")
    !{cmd}
    rm_command = f"rm {tiff_paths[k].strip()}"
    #print(f"Calling the command: {rm_command}")
    !{rm_command}

In [ ]:
tiff_paths = get_tiff_paths(paths)

<hr>
<font face="Calibri" size="4"> <b>1.3 Merge multiple frames from the same date.</b></font>
<br><br>
<font face="Calibri" size="3"><b>Create a set containing each represented date:</b></font>

In [ ]:
unique_dates = set(dates)
print(unique_dates)

<font face="Calibri" size="3"><b>Determine which dates have multiple frames. Create a dictionary with each date as a key linked to a value set as an empty string:</b></font>

In [ ]:
from collections import defaultdict

dup_dates = defaultdict(int)
for date in dates:
    dup_dates[date] += 1
    
print(dup_dates)

<font face="Calibri" size="3"><b>Update the key values in dup_paths with the string paths to all the tiffs for each date:</b></font>

In [ ]:
for pth in tiff_paths:
    date = pth.split('/')[2].split('_')[3][:8]
    if dup_dates[date] != 1:
        if type(dup_dates[date]) == int:
            dup_dates[date] = pth
        else:
            dup_dates[date] = f"{dup_dates[date]} {pth}"
    else:
        del dup_dates[date]
print(dup_dates)

<font face="Calibri" size="3"><b>Merge all the frames for each date.</b> Note that this will overwrite the first original tif and delete the remaining.</font>

In [ ]:
print(f"Performing {len(dup_dates)} merges:")
for i, dup_date in enumerate(dup_dates):
    print(f"\nMerging {i+1} of {len(dup_dates)}:\n")
    output = f"{dup_dates[dup_date].split('/')[0]}/{dup_dates[dup_date].split('/')[1]}/new{dup_dates[dup_date].split('/')[2].split(' ')[0]}"
    gdal_command = f"gdal_merge.py -o {output} {dup_dates[dup_date]}"
    print(f"\n\nCalling the command: {gdal_command}\n")
    !{gdal_command}
    for pth in dup_dates[dup_date].split(' '):
        if pth and path_exists(pth):
            os.remove(pth)
            print(f"Deleting: {pth}")

<hr>
<font face="Calibri" size="3"> <b>Verify that all duplicate dates were resolved:</b> </font>

In [ ]:
dates = get_dates(paths, 5, 0, 8)
if len(dates) == len(set(dates)):
    print(f"Duplicate dates resolved.")
else:
    print(f"Duplicate dates still present!")
print(dates)

<font face="Calibri" size="3"><b>Update the paths of the tiffs:</b></font>

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths) # uncomment to view paths

<hr>
<font face="Calibri">

<font size="5"> <b> 2. Create Subset and Stack Up Your Data </b> </font> 

<font size="3"> Now you are ready to work with your data. The next cells allow you to select an area of interest (AOI; via bounding-box corner coordinates) for your data analysis. Once selected, the AOI is being extracted and a data stack is formed.

<b>As a first step, we extract your AOI from the full frames:</b>
</font> 
</font>

In [ ]:
# Using Google Maps, get the rough bounding box for the subset
# Enter your corner coordinates below
#upper_left_x = 89.6921790
#upper_left_y = 25.1467490
#lower_right_x = 90.267578
#lower_right_y = 24.625406

#print(f"upper left x coord: {upper_left_x}\nupper left y coord: {upper_left_y}\nlower right x coord: {lower_right_x}\nlower right y coord: {lower_right_y}")

In [ ]:
# Max extents for 26 bands with 16gb memory
#upper_left_x = 89.6921790
#upper_left_y = 25.1467490
#lower_right_x = 91.05
#lower_right_y = 24.215415

#upper_left_x = 89.446831
#upper_left_y = 25.225842
#lower_right_x = 91.413732
#lower_right_y = 24.316126

upper_left_x = 29.371429
upper_left_y = 31.664799
lower_right_x = 32.402310
lower_right_y = 29.735274


print(f"upper left x coord: {upper_left_x}\nupper left y coord: {upper_left_y}\nlower right x coord: {lower_right_x}\nlower right y coord: {lower_right_y}")

<font face="Calibri" size="3"> <b>Convert the EPSG:4326 coords from Google Maps to the predominate EPSG in the data stack:</b> </font> 

In [ ]:
in_proj = Proj(init="epsg:4326")
out_proj = Proj(init=f"epsg:{stack_info['utm']}")
stack_info.update({'coords': [[None, None], [None, None]]})
stack_info['coords'][0][0], stack_info['coords'][0][1] = transform(in_proj, out_proj, 
                                                                   upper_left_x, upper_left_y)
stack_info['coords'][1][0], stack_info['coords'][1][1] = transform(in_proj, out_proj, 
                                                                   lower_right_x, lower_right_y)
print(stack_info['coords'])

<font face="Calibri" size="3"><b>Adjust the bounding box so as to avoid intersecting any pixels</b>, which then requires resampling and may slightly skew the data:</font> 

In [ ]:
for i, coords in enumerate(stack_info['coords']):
    for x, c in enumerate(coords):
        stack_info['coords'][i][x] = round(stack_info['coords'][i][x]/30)*30
print(stack_info['coords'])

<font size="3"> <b>Update the list of all the absolute paths of the tiffs:</b> </font> 

In [ ]:
tiff_paths = get_tiff_paths(paths)
#print_tiff_paths(tiff_paths) # uncomment to view paths

<font size="3"><b>Subset the tiffs and move them from the individual product directories into their own directory, /tiffs:</b></font> 

In [ ]:
new_directory('tiffs')
if path_exists('tiffs'):
    print(f"Subsetting {len(tiff_paths)} geotiffs:")
    for i, tiff_path in enumerate(tiff_paths):
        print(f"\nSubsetting {i+1} of {len(tiff_paths)} geotiffs:\n")
        _, granule_name, tiff_name = tiff_path.split('/')
        g1, g2, g3, date, g4, g5, g6 = tiff_name.split('_')
        gdal_command = f"gdal_translate -projwin {stack_info['coords'][0][0]} {stack_info['coords'][0][1]} {stack_info['coords'][1][0]} {stack_info['coords'][1][1]} -projwin_srs 'EPSG:{stack_info['utm']}' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} tiffs/{date}_{polarization}.tiff"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}

<font size="3"><b>Grab the updated paths of the images:</b></font>

In [ ]:
t_path = f"{os.getcwd()}/tiffs/"
tiff_paths = get_tiff_paths(t_path)

<font size="3"><b>Delete any subset tifs that are filled with NaNs and contain no data.</b></font>

In [ ]:
remove_nan_filled_tifs(t_path, tiff_paths)

<font size="3"><b>Update the list of dates and tiff_paths after removing NaN filled images:</b></font>

In [ ]:
print(tiff_paths)

In [ ]:
tp = f"{path}/tiffs"
tiff_paths = get_tiff_paths(tp)
subset_tif_paths = "tiffs/*.tiff"
dates = get_dates(subset_tif_paths, 0, 6, 14)
#print(dates) # uncomment to see dates
#print_tiff_paths(tiff_paths) # uncomment to see tiff_paths

<hr>
<font face="Calibri" size="3"> Now we stack up the data by creating a virtual raster table with links to all subset data files: </font>
<br><br>
<font size="3"><b>Create the virtual raster table for the subset GeoTiffs:</b></font>

In [ ]:
!gdalbuildvrt -separate raster_stack.vrt tiffs/*.tiff

<hr>
<font face="Calibri">

<font size="5"> <b> 3. Now You Can Work With Your Data </b> </font> 

<font size="3"> Now you are ready to perform time series analysis on your data stack
</font> 
</font>

<br>
<font face="Calibri" size="4"> <b> 3.1 Define Data Directory and Path to VRT </b> </font> 
<br><br>
<font face="Calibri" size="3"><b>Create a variable containing the VRT filename:</b></font>

In [ ]:
image_file = "raster_stack.vrt"

<font face="Calibri" size="3"><b>Create an index of timedelta64 data with Pandas:</b></font>

In [ ]:
stack_info.update({'time_index': pd.DatetimeIndex(dates)})

<font face="Calibri" size="3"><b>Print the bands and dates for all images in the virtual raster table (VRT):</b></font>

In [ ]:
j = 1
print(f"Bands and dates for {image_file}")
for i in stack_info['time_index']:
    print("{:4d} {}".format(j, i.date()), end=' ')
    j += 1
    if j%5 == 1:
        print()

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.2 Open Your Data Stack and Visualize Some Layers </b> </font> 

<font face="Calibri" size="3"> We will <b>open your VRT</b> and visualize some layers using Matplotlib. </font>

In [ ]:
img = gdal.Open(image_file)

<font face="Calibri" size="3"><b>Print the bands, pixels, and lines:</b></font>

In [ ]:
stack_info.update({'bands': img.RasterCount,
                  'pixels': img.RasterXSize,
                  'lines': img.RasterYSize})
for info in stack_info:
    print(f"{info}: {stack_info[info]}")

<font face="Calibri" size="3"><b>Determine the total number of pixels (bands * pixels * lines) in the VRT:</b></font>

In [ ]:
stack_size = stack_info['bands'] * stack_info['pixels'] * stack_info['lines']
print(stack_size)

<font face="Calibri" size="3"><b>If the number of pixels in the VRT is over the limit, the VRT must be tiled and processed in batches to avoid running out of memory:</b></font>

In [ ]:
from math import ceil, sqrt
tiling = False
limit = 414000000
if stack_size > limit:
    tiling = True
    tile_count = ceil(stack_size / limit)
    tile_1D = ceil(sqrt(tile_count))
    print(tile_1D)

In [ ]:
if tiling:
    tile_size_pixels = ceil(stack_info['pixels'] / tile_1D)
    tile_size_lines = ceil(stack_info['lines'] / tile_1D)

    print(f"Pixels per tile: {tile_size_pixels}")
    print(f"Lines per tile: {tile_size_lines}")
    print(f"Tile Size (including all bands): {tile_size_pixels * tile_size_lines * stack_info['bands']}")

In [ ]:
tile_dir = 'tiles'
if tiling:
    new_directory(tile_dir)
    os.chdir(tile_dir)
    for i in range(0, stack_info['pixels'], tile_size_pixels):
        for j in range(0, stack_info['lines'], tile_size_lines):
            cmd = f"gdal_translate -of VRT -srcwin {str(i)}, {str(j)}, {str(tile_size_pixels)}, {str(tile_size_lines)} {path}/{image_file} tile_{str(i)}_{str(j)}.vrt"
            !{cmd}

In [ ]:
def get_tile_paths(tile_dir):
    tile_paths = glob.glob('tile*.vrt')
    tile_paths.sort()
    return tile_paths

In [ ]:
tile_paths = get_tile_paths(f"{path}/{tile_dir}")
print(tile_paths)

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.3 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> <font color='rgba(200,0,0,0.2)'> <b>Note, that if your data were generated by HyP3, this step is not necessary!</b> HyP3 performs the full data calibration and provides you with calibrated data in power scale. </font>
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell</b>: </font> 

In [ ]:
 #caldB=20*np.log10(raster_stack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
#calPwr=np.power(10.,caldB/10.)

<hr>
<br>
<font face="Calibri" size="4"> <b> 3.4 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3">Now we are ready to create a time series animation from the calibrated SAR data.
<br><br>
<b>Write a function to mask a raster stack:</b>
</font> 

In [ ]:
def mask_raster_stack(raster_stack):
    return np.ma.masked_where(raster_stack==0, raster_stack)

<font face="Calibri" size="3"><b>Write a function to create and save a time series animation. Returns a raster stack:</b></font> 

In [ ]:
def create_time_series(gdal_img, output_filename):
    band = gdal_img.GetRasterBand(1)
    raster0 = band.ReadAsArray()
    band_number = 0 # Needed for updates
    rastr_stack = gdal_img.ReadAsArray()
    
    fig = plt.figure(figsize=(14, 8))
    ax = fig.subplots()
    ax.axis('off')
    vmin = np.percentile(rastr_stack.flatten(), 5)
    vmax = np.percentile(rastr_stack.flatten(), 95)
    
    r0dB = 20 * np.log10(raster0) - 83

    im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
    ax.set_title("{}".format(stack_info['time_index'][0].date()))

    def animate(i):
        ax.set_title("{}".format(stack_info['time_index'][i].date()))
        im.set_data(rastr_stack[i])

    # Interval is given in milliseconds
    ani = animation.FuncAnimation(fig, animate, frames=rastr_stack.shape[0], interval=400)
    
    rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation
    
    HTML(ani.to_jshtml())
    
    ani.save(output_filename, writer='pillow', fps=2)
    return rastr_stack

<font face="Calibri" size="3"><b>Create and move into a directory in which to store our plots and animations:</b></font> 

In [ ]:
os.chdir(path)
product_path = 'plots_and_animations'
new_directory(product_path)
if path_exists(product_path) and os.getcwd() != f"{path}/{product_path}":
    os.chdir(product_path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3"><b>Delete the dummy png</b> that was saved to the current working directory while generating the javascript animation in the last code cell.</font> 

In [ ]:
"""
try:
    os.remove('None0000000.png')
except FileNotFoundError:
    pass
"""

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.5 Plot the Time Series of Means Calculated Across the Subset </b> </font>

<font face="Calibri" size="3"> To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
2. compute means.
3. convert the resulting mean values into dB scale for visualization.
4. plot time series of means. </font> 
<br><br>
<font face="Calibri" size="3"> <b>Compute the means:</b> </font>

In [ ]:
def create_time_series_means(raster_stack_masked, output_filename):
    rs_means_pwr = np.mean(raster_stack_masked, axis=(1, 2))
    
    rs_means_dB = 10.*np.log10(rs_means_pwr)
    
    
    plt.rcParams.update({'font.size': 14})
    fig = plt.figure(figsize=(16, 4))
    ax1 = fig.subplots()
    window_length = len(rs_means_pwr)-1
    if window_length % 2 == 0:
        window_length -= 1
    polyorder = math.ceil(window_length*0.1)
    yhat = scipy.signal.savgol_filter(rs_means_pwr, window_length, polyorder) 
    ax1.plot(stack_info['time_index'], 
             yhat, color='red', 
             marker='o', 
             markerfacecolor='white', 
             linewidth=3, markersize=6)
    ax1.plot(stack_info['time_index'], 
             rs_means_pwr, 
             color='gray', 
             linewidth=0.5)
    plt.grid()
    ax1.set_xlabel('Date')
    ax1.set_ylabel('$\overline{\gamma^o}$ [power]')
    plt.savefig(output_filename, dpi=72, transparent='true')

In [ ]:
#rs_means_pwr = np.mean(raster_stack_masked, axis=(1, 2))

<font face="Calibri" size="3"><b>Convert resulting mean value time-series to dB scale for visualization:</b></font>

In [ ]:
#rs_means_dB = 10.*np.log10(rs_means_pwr)

<font face="Calibri" size="3"><b>Plot and save the time series of means (RCSoverTime.png):</b></font>

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.6 Calculate Coefficient of Variance </b> </font>

<font face="Calibri" size="3"> The coefficient of variance describes how much the $\sigma_{0}$ or $\gamma_{0}$ measurements in a pixel vary over time. Hence, the coefficient of variance can indicate different vegetation cover and soil moisture regimes in your area.</font> 
<br><br>
<font face="Calibri" size="3"><b>Write a function to convert our plots into GeoTiffs:</b></font> 

In [ ]:
def geotiff_from_plot(source_image, out_filename, extent, utm, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    assert "." not in out_filename, 'Error: Do not include the file extension in out_filename'
    assert type(extent) == list and len(extent) == 2 and len(extent[0]) == 2 and len(
        extent[1]) == 2, 'Error: extent must be a list in the form [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]'
    
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{utm} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        os.remove(temp)
    except FileNotFoundError:
        pass

<font face="Calibri" size="3"><b>Write a function to plot the Coefficient of Variance Map and save it as a png (Coeffvar.png):</b> </font> 

In [ ]:
def create_coeff_var_map(raster_stck, output_filename):
    test = np.var(raster_stck,0)
    mtest = np.mean(raster_stck[raster_stck.nonzero()],0)
    coeffvar = test/(mtest+0.001)

    plt.rcParams.update({'font.size': 14})
    fig = plt.figure(figsize=(13, 10))
    ax = fig.subplots()
    ax.axis('off')
    vmin = np.percentile(coeffvar.flatten(), 5)
    vmax = np.percentile(coeffvar.flatten(), 95)
    ax.set_title('Coefficient of Variance Map')

    im = ax.imshow(coeffvar, cmap='jet', vmin=vmin, vmax=vmax)
    fig.colorbar(im, ax=ax)
    plt.savefig(output_filename, dpi=300, transparent='true')
    return {'coeffvar': coeffvar, 'vmin': vmin, 'vmax': vmax}

<br>
<hr>
<font face="Calibri" size="4"> <b> 3.7 Threshold Coefficient of Variance Map </b> </font>

<font face="Calibri" size="3"> This is an example how to threshold the derived coefficient of variance map. This can be useful, e.g., to detect areas of active agriculture.</font> 
<br><br>

<font face="Calibri" size="3"><b>Write a function to plot and save the coefficient of variance histogram and CDF (thresh_coeff_var_histogram.png):</b></font> 

In [ ]:
def create_thresh_coeff_var_histogram(coeffvar, output_filename):
    plt.rcParams.update({'font.size': 14})
    fig = plt.figure(figsize=(14, 6)) # Initialize figure with a size
    ax1 = fig.add_subplot(121)  # 121 determines: 2 rows, 2 plots, first plot
    ax2 = fig.add_subplot(122)
    # Second plot: Histogram
    # IMPORTANT: To get a histogram, we first need to *flatten* 
    # the two-dimensional image into a one-dimensional vector.
    h = ax1.hist(coeffvar.flatten(), bins=200, range=(0, 0.03))
    ax1.xaxis.set_label_text('Coefficient of Variation')
    ax1.set_title('Coeffvar Histogram')
    plt.grid()
    n, bins, patches = ax2.hist(coeffvar.flatten(), bins=200, range=(0, 0.03), cumulative='True', density='True', histtype='step', label='Empirical')
    ax2.xaxis.set_label_text('Coefficient of Variation')
    ax2.set_title('Coeffvar CDF')
    plt.grid()
    plt.savefig(output_filename, dpi=72, transparent='true')
    return {'n': n, 'bins': bins}

<font face="Calibri" size="3"><b>Write a function to plot the Threshold Coefficient of Variance Map and save it as a png (Coeffvarthresh.png):</b> </font> 

In [ ]:
def create_thresh_coeff_var_map(coeffvar, output_filename, n_bins):
    plt.rcParams.update({'font.size': 14})
    outind = np.where(n_bins['n'] > 0.85)
    threshind = np.min(outind)
    thresh = n_bins['bins'][threshind]
    coeffvarthresh = np.copy(coeffvar)
    coeffvarthresh[coeffvarthresh < thresh] = 0
    coeffvarthresh[coeffvarthresh > 0.1] = 0
    fig = plt.figure(figsize=(13, 10))
    ax = fig.subplots()
    ax.axis('off')
    vmin = np.percentile(coeffvar.flatten(), 5)
    vmax = np.percentile(coeffvar.flatten(), 95)
    ax.set_title(r'Thresholded Coeffvar Map [$\alpha=95%$]')
    im = ax.imshow(coeffvarthresh, cmap='jet', vmin=vmin, vmax=vmax)
    bar = fig.colorbar(im, ax=ax)
    plt.savefig(output_filename, dpi=300, transparent='true')
    return {'coeffvarthresh': coeffvarthresh, 'vmin': vmin, 'vmax': vmax}

<font face="Calibri" size="3"><b>Write a function to create a stack_info dictionary for a tile:</b> </font> 

In [ ]:
def get_tile_stack_info(stack_info):
    tile_stack_info = deepcopy(stack_info)
    tile_info = (gdal.Info(img, options = ['-json']))
    tile_corner_coords = (json.loads(tile_info))['cornerCoordinates']
    
    tile_stack_info['coords'][0][0] = tile_corner_coords['upperLeft'][0]
    tile_stack_info['coords'][0][1] = tile_corner_coords['upperLeft'][1]
    tile_stack_info['coords'][1][0] = tile_corner_coords['lowerRight'][0]
    tile_stack_info['coords'][1][1] = tile_corner_coords['lowerRight'][1]
   
    tile_stack_info.update({'bands': img.RasterCount,
                  'pixels': img.RasterXSize,
                  'lines': img.RasterYSize})
    
    return tile_stack_info

<font face="Calibri" size="3"><b>Write a function to pickle data:</b></font>

In [ ]:
def ferment(cucumber, output_filename):
    pf = open(output_filename, 'ab')
    pickle.dump(cucumber, pf)
    pf.close()

In [ ]:
from copy import deepcopy

for tile in tile_paths:
    img = gdal.Open(f"{path}/{tile_dir}/{tile}")
    tile_id = f"{tile.split('_')[1]}_{tile.split('_')[2].split('.')[0]}"
    
    tile_stack_info = get_tile_stack_info(stack_info)
    
    print(f"Tile: {tile_id}")
    #print(f"stack_info coords: {stack_info['coords']}")
    #print(tile_stack_info['coords'])
          
    ferment(tile_stack_info, f"{path}/pickles/pickled_tile_stack_info_{tile_id}")
    
    raster_stack = create_time_series(img, f"animation_{tile_id}.gif")
    plt.close('all')
    gc.collect()
    ferment(raster_stack, f"{path}/pickles/pickled_tile_raster_stack_{tile_id}")
           
    raster_stack_masked = mask_raster_stack(raster_stack)
    create_time_series_means(raster_stack_masked, f"RCSoverTime_{tile_id}.png")
    plt.close('all')
    gc.collect()
    ferment(raster_stack_masked, f"{path}/pickles/pickled_masked_tile_raster_stack_{tile_id}")
    
    coeffvar_data = create_coeff_var_map(raster_stack, f"Coeffvar_{tile_id}.png")
    plt.close('all')
    gc.collect()
    geotiff_from_plot(coeffvar_data['coeffvar'], f"Coeffvar_{tile_id}", tile_stack_info['coords'], tile_stack_info['utm'], cmap='jet', vmin=coeffvar_data['vmin'], vmax=coeffvar_data['vmax'])
    
    n_b = create_thresh_coeff_var_histogram(coeffvar_data['coeffvar'], f"thresh_coeff_var_histogram_{tile_id}.png")
    
    coeffvarthresh_data = create_thresh_coeff_var_map(coeffvar_data['coeffvar'], f"Coeffvarthresh_{tile_id}.png", n_b)
    plt.close('all')
    gc.collect()
    geotiff_from_plot(coeffvarthresh_data['coeffvarthresh'], 'Coeffvarthresh', tile_stack_info['coords'], tile_stack_info['utm'], cmap='jet', vmin=coeffvarthresh_data['vmin'], vmax=coeffvarthresh_data['vmax'])

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - March 2019 </i>
</font>